In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Transformations in Code
todo

## Xform Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D96 Xform Large.jpg" style="width: 600px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P17.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P18.jpg" style="width: 200px; display: inline;">

## A Library of Spatial Transformations

### Change of Basis

\begin{align}
     {\large M_{\vec{s_{1}},\vec{s_{2}},\vec{s_{3}} \to \vec{e_{1}},\vec{e_{2}},\vec{e_{3}}} } = 
    \begin{bmatrix}
        s_{1x} & s_{2x} & s_{3x} & 0 \\
        s_{1y} & s_{2y} & s_{3y} & 0 \\
        s_{1z} & s_{2z} & s_{3z} & 0 \\  
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
\end{align}

\begin{align}
     {\large M_{\vec{e_{1}},\vec{e_{2}},\vec{e_{3}} \to \vec{t_{1}},\vec{t_{2}},\vec{t_{3}} } } = 
    \begin{bmatrix}
        t_{1x} & t_{1x} & t_{1x} & 0 \\
        t_{2y} & t_{2y} & t_{2y} & 0 \\
        t_{3z} & t_{3z} & t_{3z} & 0 \\  
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
\end{align}

\begin{align}
    \begin{bmatrix}
        1 & 0 & 0 & o_{tx} \\
        0 & 1 & 0 & o_{ty} \\
        0 & 0 & 1 & o_{tz} \\    
        0 & 0 & 0 & 1 \\
    \end{bmatrix}
     \times
    {\large M_{\vec{e_{1}},\vec{e_{2}},\vec{e_{3}} \to \vec{t_{1}},\vec{t_{2}},\vec{t_{3}} } } 
     \times
     {\large M_{\vec{s_{1}},\vec{s_{2}},\vec{s_{3}} \to \vec{e_{1}},\vec{e_{2}},\vec{e_{3}}} }
     \times
    \begin{bmatrix}
        1 & 0 & 0 & -o_{sx} \\
        0 & 1 & 0 & -o_{sy} \\
        0 & 0 & 1 & -o_{sz} \\    
        0 & 0 & 0 & 1 \\
    \end{bmatrix}     
\end{align}

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.09.P27.jpg" style="width: 200px; display: inline;">